In [171]:
import numpy as np
import pandas as pd
import string
import re

In [172]:
with open ('invoices.txt', mode = 'r', encoding = 'utf8', errors = 'ignore') as f:
    text = f.read()

In [173]:
# text.split('\n')
data = list(map(lambda x: x.split('\t'),text.split('\n')))

data

[['id', 'text', 'tag'],
 ['1.png', 'Company Name', 'O'],
 ['1.png', 'INVOICE', 'O'],
 ['1.png', '[Street Accress]', 'O'],
 ['1.png', '[City, ST  ZIP]', 'O'],
 ['1.png', 'DATE', 'O'],
 ['1.png', '12-9-2019', 'DATE'],
 ['1.png', 'INVOICE', 'O'],
 ['1.png', '[123456]', 'NUMB'],
 ['1.png', 'CUSTOMER ID', 'O'],
 ['1.png', '[123]', 'O'],
 ['1.png', 'Website: somecomain.com', 'O'],
 ['1.png', 'DUE DATE', 'O'],
 ['1.png', '1-8-2020', 'O'],
 ['1.png', 'BILL TO', 'O'],
 ['1.png', '[Name]', 'O'],
 ['1.png', '[Company Name]', 'O'],
 ['1.png', '[Street Accres]', 'O'],
 ['1.png', '[City, ST  ZIP]', 'O'],
 ['1.png', '[Phone]', 'O'],
 ['1.png', 'DESCRIPTION', 'O'],
 ['1.png', 'TAXED', 'O'],
 ['1.png', 'AMOUNT', 'O'],
 ['1.png', '[Service Fee_', 'O'],
 ['1.png', '230.00', 'O'],
 ['1.png', '[Labor:', 'O'],
 ['1.png', 'hours at S75/hr]', 'O'],
 ['1.png', '375.00', 'O'],
 ['1.png', '[Parts]', 'O'],
 ['1.png', '345.00', 'O'],
 ['1.png', 'Subtotal', 'O'],
 ['1.png', '950.00', 'O'],
 ['1.png', 'Taxabl e', 'O

In [174]:
df = pd.DataFrame(data[1:],columns=data[0])
df.dropna(inplace=True)
df

,id,text,tag
0,1.png,Company Name,O
1,1.png,INVOICE,O
2,1.png,[Street Accress],O
3,1.png,"[City, ST ZIP]",O
4,1.png,DATE,O
...,...,...,...
4497,96.png,R330.00,O
4498,96.png,Sub Total: R6 156.53,O
4499,96.png,Please pay within 14 days_,O
4500,96.png,Total Tax: R923.47,O


In [175]:
# Cleaning Text
# remove whitww space and unwanted special characters


In [176]:
whitespace = string.whitespace
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
tableWhitespace = str.maketrans('', '', whitespace)
tablePunctuation = str.maketrans('', '', punctuation)
def cleanText(txt):
    text= str(txt)
    text = text.lower()
    removewhitespaces = text.translate(tableWhitespace)
    removepunctuation = text.translate(tablePunctuation)
    return str(removepunctuation)

In [177]:
df['text'] = df['text'].apply(cleanText)

In [178]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

C:\Users\inass.elmadhoun\AppData\Local\Temp\ipykernel_7352\3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [179]:
dataClean.head(10)

,id,text,tag
0,1.png,company name,O
1,1.png,invoice,O
2,1.png,street accress,O
3,1.png,city st zip,O
4,1.png,date,O
5,1.png,1292019,DATE
6,1.png,invoice,O
7,1.png,123456,NUMB
8,1.png,customer id,O
9,1.png,123,O


In [181]:
group = dataClean.groupby(by='id')
invoices = group.groups.keys()
allInvoiceData = []
for invoice in invoices:
    invoiceData = []
    grouparray = group.get_group(invoice)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1
        start = end
        end = start + stringLength
        
        if label != 'O':
            annot = (start, end-1, label)
            annotations['entities'].append(annot)
        content = content + text + ' '
    invoiceData = (content, annotations)
    allInvoiceData.append(invoiceData)

In [182]:
len(allInvoiceData)

104

In [183]:
# Split data in train data en testing data 
import random
random.shuffle(allInvoiceData)
len(allInvoiceData)

104

In [184]:
trainData = allInvoiceData[:50]
testData = allInvoiceData[85:]

In [185]:
import pickle

In [186]:
pickle.dump(trainData,open('./data/trainData.pickle', mode='wb'))
pickle.dump(testData,open('./data/testData.pickle', mode='wb'))